In [1]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor

pygame 2.6.0 (SDL 2.28.4, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from custom_env import CustomEnv
from wrappers import FullyObsSB3MLPWrapper
from callbacks import EvalSaveCallback

C:\Users\13769\anaconda3\envs\mini-grid-generation\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\13769\anaconda3\envs\mini-grid-generation\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\13769\anaconda3\envs\mini-grid-generation\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
# Define the session directory
session_dir = r"./experiments/small_corridor"
os.makedirs(session_dir, exist_ok=True)

# Set the log directory within the session directory
log_dir = os.path.join(session_dir, "logs")
os.makedirs(log_dir, exist_ok=True)

# Set training parameters
max_train_episode_steps = 1000
max_eval_episode_steps = 500
total_timesteps = int(1e5)
eval_deterministic = False
eval_freq = int(5e3)
n_eval_episodes = 10
model_save_path = os.path.join(session_dir, "latest_model")

In [4]:
# Initialize the environment and wrapper
train_env = CustomEnv(
    txt_file_path='simple_test_corridor.txt',
    display_size=6,
    display_mode="random",
    random_rotate=True,
    random_flip=True,
    custom_mission="Find the key and open the door.",
    max_steps=max_train_episode_steps
)

# Wrap the environment with FullyObsSB3MLPWrapper
train_env = FullyObsSB3MLPWrapper(train_env)

# Use DummyVecEnv and VecMonitor to create and wrap the environment
train_env = DummyVecEnv([lambda: train_env])  # Vectorize the environment
train_env = VecMonitor(train_env)  # Track episode statistics such as rewards

In [5]:
# Initialize the environment and wrapper
eval_env = CustomEnv(
    txt_file_path='simple_test_corridor.txt',
    display_size=6,
    display_mode="middle",
    random_rotate=False,
    random_flip=False,
    custom_mission="Find the key and open the door.",
    max_steps=max_eval_episode_steps
)

# Wrap the environment with FullyObsSB3MLPWrapper
eval_env = FullyObsSB3MLPWrapper(eval_env)

# Use DummyVecEnv and VecMonitor to create and wrap the environment
eval_env = DummyVecEnv([lambda: eval_env])  # Vectorize the environment
eval_env = VecMonitor(eval_env)  # Track episode statistics such as rewards

In [6]:
# Load or create a new model
if os.path.exists(f"{model_save_path}.zip"):
    model = PPO.load(model_save_path, env=train_env)
    print("Loaded model from saved path.")
else:
    model = PPO("MlpPolicy", train_env, verbose=1)
    print("Initialized new model.")

Using cuda device
Initialized new model.


In [7]:
# Create EvalSaveCallback
eval_save_callback = EvalSaveCallback(
    eval_env=eval_env,
    log_dir=log_dir,
    eval_freq=eval_freq,
    n_eval_episodes=n_eval_episodes,
    deterministic=eval_deterministic,
    verbose=1,
)

In [8]:
# Train the model and log performance with the custom callback
model.learn(total_timesteps=total_timesteps, callback=eval_save_callback, progress_bar=True)

Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 139      |
|    ep_rew_mean     | 0.875    |
| time/              |          |
|    fps             | 415      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 151         |
|    ep_rew_mean          | 0.864       |
| time/                   |             |
|    fps                  | 369         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010126465 |
|    clip_fraction        | 0.115       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.94       |
|    explained_variance   | 0.0591      |
|    learning_rate        | 0.

Evaluating model...

Evaluation: Mean reward: 0.81 +/- 0.16

Saved latest model to ./experiments/small_corridor\logs\latest_model.zip

...

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 4.06       |
|    ep_rew_mean          | 0.996      |
| time/                   |            |
|    fps                  | 280        |
|    iterations           | 47         |
|    time_elapsed         | 343        |
|    total_timesteps      | 96256      |
| train/                  |            |
|    approx_kl            | 0.03369914 |
|    clip_fraction        | 0.122      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.11      |
|    explained_variance   | 0.605      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0579    |
|    n_updates            | 460        |
|    policy_gradient_loss | -0.032     |
|    value_loss           | 0.000264   |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 8.38       |
|    ep_rew_mean

Evaluating model...

Evaluation: Mean reward: 0.99 +/- 0.00

Saved latest model to ./experiments/small_corridor\logs\latest_model.zip

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 4          |
|    ep_rew_mean          | 0.996      |
| time/                   |            |
|    fps                  | 279        |
|    iterations           | 49         |
|    time_elapsed         | 359        |
|    total_timesteps      | 100352     |
| train/                  |            |
|    approx_kl            | 0.14872073 |
|    clip_fraction        | 0.41       |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.323     |
|    explained_variance   | 0.359      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.104     |
|    n_updates            | 480        |
|    policy_gradient_loss | 0.0612     |
|    value_loss           | 0.000941   |
----------------------------------------
